# The TinyALU TLM Testbench

In [1]:
from pyuvm import *
from tb_pkg import *
import random
import time



class AluSeqItem(uvm_sequence_item):

    def __init__(self, name, aa=0, bb=0, op=Ops.ADD):
        super().__init__(name)
        self.A = aa
        self.B = bb
        self.op = Ops(op)

    def __eq__(self, other):
        same = self.A == other.A and self.B == other.B and self.op == other.op
        return same

    def __str__(self):
        return f"{self.get_name()} : A: 0x{self.A:02x} OP: {self.op.name} ({self.op.value}) B: 0x{self.B:02x}"
    
    def randomize(self):
        self.A = random.randint(0, 255)
        self.B = random.randint(0, 255)
        self.op = random.choice(list(Ops))
        

class AluSeq(uvm_sequence):
    def body(self):
        for _ in range(5):
            cmd_tr = AluSeqItem("cmd_tr")
            self.start_item(cmd_tr) 
            cmd_tr.randomize()
            self.finish_item(cmd_tr) 
            

class Driver(uvm_driver):
    def build_phase(self):
        self.proxy = self.cdb_get("PROXY")

    def run_phase(self):
        while not ObjectionHandler().run_phase_complete():
            command = self.seq_item_port.get_next_item()
            self.proxy.send_op(command.A, command.B, command.op)
            self.logger.debug(f"Sent command: {command}")
            time.sleep(0.5)
            self.seq_item_port.item_done()


class Coverage(uvm_subscriber):
    
    def end_of_elaboration_phase(self):
        self.cvg = set()
    
    def write(self, cmd):
        (_, _, op) = cmd
        self.cvg.add(op)

    def check_phase(self):
        if len(set(Ops) - self.cvg) > 0:
            self.logger.error(f"Functional coverage error. Missed: {set(Ops)-self.cvg}")


class Scoreboard(uvm_component):  

    def build_phase(self):
        self.cmd_fifo = uvm_tlm_analysis_fifo("cmd_fifo", self)
        self.result_fifo = uvm_tlm_analysis_fifo("result_fifo", self)
        self.cmd_get_port = uvm_get_port("cmd_get_port", self)    
        self.result_get_port = uvm_get_port("result_get_port", self)
        self.cmd_export = self.cmd_fifo.analysis_export
        self.result_export = self.result_fifo.analysis_export

    def connect_phase(self):
        self.cmd_get_port.connect(self.cmd_fifo.get_export)
        self.result_get_port.connect(self.result_fifo.get_export)

    def check_phase(self):
        while self.result_get_port.can_get():
            _, actual_result = self.result_get_port.try_get()
            cmd_success, (A, B, op_numb) = self.cmd_get_port.try_get()
            if not cmd_success:
                self.logger.critical(f"result {actual_result} had no command")
            else:
                op = Ops(op_numb)
                predicted_result = ModelProxy.alu_op(A, B, op)
                if predicted_result == actual_result:
                    self.logger.info(f"PASSED: 0x{A:02x} {op.name} 0x{B:02x} ="
                                     f" 0x{actual_result:04x}")
                else:
                    self.logger.error(f"FAILED: 0x{A:02x} {op.name} 0x{B:02x} "
                                      f"= 0x{actual_result:04x} expected 0x{predicted_result:04x}")


class Monitor(uvm_component):
    def __init__(self, name, parent, method_name):
        super().__init__(name, parent)
        self.method_name = method_name
    
    def build_phase(self):
        self.proxy = self.cdb_get("PROXY")
        self.ap = uvm_analysis_port("ap", self)

    def run_phase(self):
        while not ObjectionHandler().run_phase_complete():
            get_method = getattr(self.proxy, self.method_name)
            datum = get_method()
            self.ap.write(datum)    


class AluEnv(uvm_env):

    def build_phase(self):
        self.cmd_mon = Monitor("cmd_mon", self, "get_cmd")
        self.result_mon = Monitor("result_mon", self, "get_result")
        self.scoreboard = Scoreboard("scoreboard", self)
        self.coverage = Coverage("coverage", self)
        self.driver = Driver("driver", self)
        self.seqr = uvm_sequencer("seqr", self)
        ConfigDB().set(None, "*", "SEQR", self.seqr)
        ConfigDB().set(None, "*", "CVG", self.coverage)
        self.proxy = ModelProxy("PROXY", self)
        ConfigDB().set(None, "*", "PROXY", self.proxy)        
        
    def connect_phase(self):
        self.cmd_mon.ap.connect(self.scoreboard.cmd_export)
        self.cmd_mon.ap.connect(self.coverage)
        self.result_mon.ap.connect(self.scoreboard.result_export)
        self.driver.seq_item_port.connect(self.seqr.seq_item_export)

class AluTest(uvm_test):
    def build_phase(self):
        self.env = AluEnv.create("env", self)

    def run_phase(self):
        self.raise_objection()
        seqr = ConfigDB().get(self, "", "SEQR")
        seq = AluSeq("seq")
        seq.start(seqr)
        time.sleep(1)
        self.drop_objection()

    def end_of_elaboration_phase(self):
        self.set_logging_level_hier(logging.DEBUG)
        
uvm_root().run_test("AluTest")
        

DEBUG: <ipython-input-1-705b06c2b54a>(46)[uvm_test_top.env.driver]: Sent command: cmd_tr : A: 0xb1 OP: MUL (4) B: 0x17
DEBUG: <ipython-input-1-705b06c2b54a>(46)[uvm_test_top.env.driver]: Sent command: cmd_tr : A: 0xf0 OP: XOR (3) B: 0x76
DEBUG: <ipython-input-1-705b06c2b54a>(46)[uvm_test_top.env.driver]: Sent command: cmd_tr : A: 0x43 OP: XOR (3) B: 0x79
DEBUG: <ipython-input-1-705b06c2b54a>(46)[uvm_test_top.env.driver]: Sent command: cmd_tr : A: 0x33 OP: AND (2) B: 0x7c
DEBUG: <ipython-input-1-705b06c2b54a>(46)[uvm_test_top.env.driver]: Sent command: cmd_tr : A: 0xf6 OP: ADD (1) B: 0x55
INFO: <ipython-input-1-705b06c2b54a>(89)[uvm_test_top.env.scoreboard]: PASSED: 0xb1 MUL 0x17 = 0x0fe7
INFO: <ipython-input-1-705b06c2b54a>(89)[uvm_test_top.env.scoreboard]: PASSED: 0xf0 XOR 0x76 = 0x0086
INFO: <ipython-input-1-705b06c2b54a>(89)[uvm_test_top.env.scoreboard]: PASSED: 0x43 XOR 0x79 = 0x003a
INFO: <ipython-input-1-705b06c2b54a>(89)[uvm_test_top.env.scoreboard]: PASSED: 0x33 AND 0x7c = 0x00